In [1]:
import os

root_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))
os.chdir(root_dir)

In [2]:
# import io
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
sns.set_style("whitegrid")
sns.set_context("paper", font_scale=1.4)
# sns.set_palette("Set2")
import functions
from IsoelectricPoint import IsoelectricPoint

from multiprocessing import Pool
from functools import partial

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef
# from Bio.SeqUtils.ProtParam import ProteinAnalysis

# from ipywidgets import interact
# import ipywidgets as widgets
from scipy.signal import savgol_filter

%load_ext autoreload
%autoreload 2



# Kyte & Doolittle index of hydrophobicity
# J. Mol. Biol. 157:105-132(1982).
kd = {"A": 1.8, "R": -4.5, "N": -3.5, "D": -3.5, "C": 2.5,
      "Q": -3.5, "E": -3.5, "G": -0.4, "H": -3.2, "I": 4.5,
      "L": 3.8, "K": -3.9, "M": 1.9, "F": 2.8, "P": -1.6,
      "S": -0.8, "T": -0.7, "W": -0.9, "Y": -1.3, "V": 4.2}

#Sort kd
kd = {k: v for k, v in sorted(kd.items(), key=lambda item: item[1])}

# Flexibility
# Normalized flexibility parameters (B-values), average
# Vihinen M., Torkkila E., Riikonen P. Proteins. 19(2):141-9(1994).
flexibilities_vih = {"A": 0.984, "C": 0.906, "E": 1.094, "D": 1.068,
"G": 1.031, "F": 0.915, "I": 0.927, "H": 0.950,
"K": 1.102, "M": 0.952, "L": 0.935, "N": 1.048,
"Q": 1.037, "P": 1.049, "S": 1.046, "R": 1.008,
"T": 0.997, "W": 0.904, "V": 0.931, "Y": 0.929}




def flex(seq):
    seq = seq.replace('U', 'C')
    if len(seq) == 0:
        return 0
    else:
        return [flexibilities_vih[i] for i in seq]

def gravy(seq):
    seq = seq.replace('U', 'C')
    if len(seq) == 0:
        return 0
    else:
        return np.mean([kd[i] for i in seq])

def hydrop(seq):
    seq = seq.replace('U', 'C')
    return [kd[i] for i in seq]

def filt(seq, w=15, p=2):
    seq = seq.replace('U', 'C')
    aa = hydrop(seq)
    return savgol_filter(aa, w, p)

aa = [k for k, v in kd.items()]


def progress(iteration, total, message=None):
    '''Simple progressbar
    '''
    if message is None:
        message = ''
    bars_string = int(float(iteration) / float(total) * 50.)
    print("\r|%-50s| %d%% (%s/%s) %s "% ('█'*bars_string+ "░" * \
                                     (50 - bars_string), float(iteration)/\
                                     float(total) * 100, iteration, total, \
                                     message), end='\r', flush=True)

    if iteration == total:
        print('\nCompleted!')


# def isoelectric(seq):
#     protparam = ProteinAnalysis(seq)
#     return protparam.isoelectric_point()


In [115]:
# isoelectric('PETER')

In [3]:
from itertools import chain, combinations

def powerset(iterable):
    # To make all combination of features
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))


# features_list = ['Hydrophobicity', 'SWI', 'Flexibility', 'Helix', 'Isoelectric Point']

def make_feat(seq, comb):
    hydro = hydrop(seq)
    swi = functions.solubility_score(seq)
    fl = flex(seq)
    hel = [seq.count(i) for i in 'VIYFWL']
    turn = [seq.count(i) for i in 'NPGS']
#     iso = [isoelectric(seq)]
    iso = [IsoelectricPoint(seq).pi()]
    feats = [hydro, swi, fl, hel, turn, iso]
    
    selected = []
    for i in comb:
        selected += feats[i]
    return  selected

features_list = ['Hydrophobicity', 'SWI', 'Flexibility', 'Helix', 'Turn', 'Isoelectric Point']
all_feats = [i for i in powerset([0, 1, 2, 3, 4, 5])][1:]
all_feats


[(0,),
 (1,),
 (2,),
 (3,),
 (4,),
 (5,),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (2, 3),
 (2, 4),
 (2, 5),
 (3, 4),
 (3, 5),
 (4, 5),
 (0, 1, 2),
 (0, 1, 3),
 (0, 1, 4),
 (0, 1, 5),
 (0, 2, 3),
 (0, 2, 4),
 (0, 2, 5),
 (0, 3, 4),
 (0, 3, 5),
 (0, 4, 5),
 (1, 2, 3),
 (1, 2, 4),
 (1, 2, 5),
 (1, 3, 4),
 (1, 3, 5),
 (1, 4, 5),
 (2, 3, 4),
 (2, 3, 5),
 (2, 4, 5),
 (3, 4, 5),
 (0, 1, 2, 3),
 (0, 1, 2, 4),
 (0, 1, 2, 5),
 (0, 1, 3, 4),
 (0, 1, 3, 5),
 (0, 1, 4, 5),
 (0, 2, 3, 4),
 (0, 2, 3, 5),
 (0, 2, 4, 5),
 (0, 3, 4, 5),
 (1, 2, 3, 4),
 (1, 2, 3, 5),
 (1, 2, 4, 5),
 (1, 3, 4, 5),
 (2, 3, 4, 5),
 (0, 1, 2, 3, 4),
 (0, 1, 2, 3, 5),
 (0, 1, 2, 4, 5),
 (0, 1, 3, 4, 5),
 (0, 2, 3, 4, 5),
 (1, 2, 3, 4, 5),
 (0, 1, 2, 3, 4, 5)]

In [118]:
df = pd.read_pickle('results/toxin_nontoxin.pkl.gz')
df = df.sample(frac=1, random_state=12345)

In [119]:
df.Label.value_counts()

0    1738
1     261
Name: Label, dtype: int64

In [120]:
def all_features_combination(length):
    

    all_cross_val_df = []

    for co in all_feats:
        df_ = df.copy()
        df_['Feat'] = df_['Protein'].str[:length].apply(lambda x: make_feat(x, co))

        dfs = np.array_split(df_, 5)


        cv_results = pd.DataFrame(columns=['Test', 'Train', 'Classifier', \
                                          'Precision', 'Recall', 'PR_AUC', \
                                          'FPR', 'TPR', 'ROC_AUC', 'MCC', 'Threshold'], index=np.arange(1, 6))
        ll = np.arange(5)
        mccs = []

        for j in range(5):
            test_idx = j
            train_idx = list(set(ll) - set([j]))

            test = dfs[j].copy()
            train = pd.concat([dfs[i] for i in train_idx]).copy()

            train = train.reset_index(drop=True)
            train = train.sample(frac=1, random_state=12345).copy()

            #Making random forest



            X_train = [i for i in train['Feat']]
            y_train = train['Label'].values

            X_test = [i for i in test['Feat']]
            y_test = test['Label'].values


            cv_results.loc[j+1]['Train'] = [X_train, y_train]
            cv_results.loc[j+1]['Test'] = [X_test, y_test]

            clf = RandomForestClassifier(random_state=12345)
            clf.fit(X_train, y_train)

            cv_results.loc[j+1]['Classifier'] = clf


            preds = [i[1] for i in clf.predict_proba(X_test)]
            dfs[j]['SP_Prediction'] = preds

            #Precision, recall
        #     precision, recall, _ = precision_recall_curve(y_test, preds, )
        #     pr_auc = auc(recall, precision)

        #     cv_results.loc[j+1][['Precision', 'Recall', 'PR_AUC']] = [precision], [recall], pr_auc


        #     cv_results.loc[j+1][['FPR', 'TPR', 'ROC_AUC']] = [fpr], [tpr], roc_auc

            preds = [i[1] for i in clf.predict_proba(X_test)]
            dfs[j]['SP_Prediction'] = preds



            mcc = []

            for i, v in enumerate(np.arange(0.01, 0.99, 0.01)):
                y_pred = [0 if x <= v else 1 for x in preds]
                mcc.append(matthews_corrcoef(y_test, y_pred))

            mccs.append(max(mcc))
            cv_results.loc[j+1]['Threshold'] = np.arange(0.01, 0.99, 0.01)[np.argmax(mcc)]
#             functions.progress(j+1, 5)

        cv_results['MCC'] = mccs    
        all_cross_val_df.append(cv_results)
    
    return all_cross_val_df




In [121]:
# for i, v in enumerate(range(15, 31, 1)):
#     print(v)

In [122]:
a = []

for i, v in enumerate(range(15, 31, 1)):
    m = all_features_combination(v)
    a.append(m)

    functions.progress(i+1, 16)

/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhand

/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhand

/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhand

/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhand

/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhand

...

/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhand

/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhand

/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


|██████████████████████████████████████████████████| 100% (16/16)  
Completed!


/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [ ]:
import pickle

with open('results/toxin_features_length.pkl', 'wb') as f:
    pickle.dump(a, f)

In [123]:
a[0][0]

,Test,Train,Classifier,Precision,Recall,PR_AUC,FPR,TPR,ROC_AUC,MCC,Threshold
1,"[[[1.9, -4.5, 1.8, 4.2, 4.2, 4.2, 3.8, 3.8, 3....","[[[1.9, -3.5, -3.9, 4.2, 3.8, 1.8, 3.8, 4.2, 3...","(DecisionTreeClassifier(max_features='auto', r...",NaN,NaN,NaN,NaN,NaN,NaN,0.566998,0.32
2,"[[[1.9, -3.9, 4.2, -0.8, -0.7, 4.5, 2.8, 1.8, ...","[[[1.9, -3.5, -3.9, 4.2, 3.8, 1.8, 3.8, 4.2, 3...","(DecisionTreeClassifier(max_features='auto', r...",NaN,NaN,NaN,NaN,NaN,NaN,0.630520,0.24
3,"[[[1.9, 4.5, 4.5, -3.9, -0.7, 3.8, 3.8, 1.8, -...","[[[1.9, -3.5, -3.9, 4.2, 3.8, 1.8, 3.8, 4.2, 3...","(DecisionTreeClassifier(max_features='auto', r...",NaN,NaN,NaN,NaN,NaN,NaN,0.726994,0.42
4,"[[[1.9, -3.5, -4.5, -0.4, 4.2, -1.6, 2.8, -4.5...","[[[1.9, -3.5, -3.9, 4.2, 3.8, 1.8, 3.8, 4.2, 3...","(DecisionTreeClassifier(max_features='auto', r...",NaN,NaN,NaN,NaN,NaN,NaN,0.695129,0.49
5,"[[[1.9, -3.5, 2.5, 3.8, -3.9, 4.5, 2.5, -0.4, ...","[[[1.9, -4.5, 3.8, -3.2, -3.2, 3.8, 3.8, 3.8, ...","(DecisionTreeClassifier(max_features='auto', r...",NaN,NaN,NaN,NaN,NaN,NaN,0.662196,0.31


In [124]:
final_results = []

for x, y in enumerate(range(15, 31, 1)):
    for i in range(len(all_feats)):
        max_mcc = max([np.median(a[x][i]['MCC']) for i in range(len(all_feats))])
        if np.median(a[x][i]['MCC']) == max_mcc:
#             print(x, y)
            final_results.append([all_feats[i], y, max_mcc])

max_median_MCC = np.max([i[2] for i in final_results])

for i, v in enumerate(final_results):
    if v[2] == max_median_MCC:
        print('Max median MCC = {} happened at length {}.\n '.format(max_median_MCC, v[1]))
        print('This was for features {}.'.format([features_list[p] for p in v[0]]))

Max median MCC = 0.7405575627922645 happened at length 23.
 
This was for features ['Hydrophobicity', 'SWI', 'Flexibility', 'Turn'].


In [126]:
len(final_results)

17

In [127]:
cols = ['Features', 'N-terminal length', 'Max. of median MCC']

# 33 because two repeats

final_results_df = pd.DataFrame(columns=cols, index=list(range(15, 15+ len(final_results), 1)))
for x, y in enumerate(range(15, 15+ len(final_results), 1)):
    final_results_df.loc[y]['Features'] = [features_list[i] for i in final_results[x][0]]
    final_results_df.loc[y]['N-terminal length'] = final_results[x][1]
    final_results_df.loc[y]['Max. of median MCC'] = np.round(final_results[x][2], 3)
final_results_df.round(4)

,Features,N-terminal length,Max. of median MCC
15,"[Hydrophobicity, SWI, Flexibility, Helix, Turn...",15,0.727
16,"[Hydrophobicity, SWI, Flexibility, Turn, Isoel...",16,0.731
17,"[Hydrophobicity, SWI, Helix, Turn]",17,0.716
18,"[Hydrophobicity, SWI, Isoelectric Point]",18,0.726
19,"[Hydrophobicity, SWI, Turn, Isoelectric Point]",19,0.727
20,"[SWI, Flexibility, Helix, Turn]",20,0.727
21,"[Hydrophobicity, SWI, Flexibility]",21,0.718
22,"[Hydrophobicity, SWI, Isoelectric Point]",22,0.717
23,"[Hydrophobicity, SWI, Flexibility, Turn]",23,0.741
24,"[Hydrophobicity, SWI]",24,0.715
